In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from langdetect import detect
from pyspark.sql.functions import when, udf, col
import pyspark as ps
import datetime
from pyspark.sql import functions as f
from pyspark.sql import types as t
from pyspark.sql.types import DateType
from sparknlp.pretrained import PretrainedPipeline

In [2]:
spark = SparkSession.builder \
    .appName("Redit Summarization")\
    .master("yarn")\
    .config("spark.driver.spark.driver.cores","4")\
    .config("spark.executor.memoryOverhead","2048")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.2.3,org.apache.hadoop:hadoop-aws:2.7.3")\
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5cee138f-20f7-457b-9b74-87c314af1770;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;3.2.3 in central
	found com.typesafe#config;1.4.1 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.603 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastu

:: retrieving :: org.apache.spark#spark-submit-parent-5cee138f-20f7-457b-9b74-87c314af1770
	confs: [default]
	0 artifacts copied, 84 already retrieved (0kB/40ms)
2022-04-24 13:55:04,130 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-04-24 13:55:06,275 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2022-04-24 13:55:08,257 WARN yarn.Client: Failed to cleanup staging dir hdfs://namenode:9000/user/ubuntu/.sparkStaging/application_1644567623435_0205
org.apache.hadoop.hdfs.server.namenode.SafeModeException: Cannot delete /user/ubuntu/.sparkStaging/application_1644567623435_0205. Name node is in safe mode.
Resources are low on NN. Please add or free up more resourcesthen turn off safe mode manually. NOTE:  If you turn off safe mode before a

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.hadoop.hdfs.server.namenode.SafeModeException: Cannot create directory /user/ubuntu/.sparkStaging/application_1644567623435_0205. Name node is in safe mode.
Resources are low on NN. Please add or free up more resourcesthen turn off safe mode manually. NOTE:  If you turn off safe mode before adding resources, the NN will immediately return to safe mode. Use "hdfs dfsadmin -safemode leave" to turn safe mode off. NamenodeHostName:namenode
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.newSafemodeException(FSNamesystem.java:1476)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.checkNameNodeSafeMode(FSNamesystem.java:1463)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.mkdirs(FSNamesystem.java:3232)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.mkdirs(NameNodeRpcServer.java:1145)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServ

In [ ]:
from sparknlp.pretrained import PretrainedPipeline, LanguageDetectorDL, PipelineModel

In [ ]:
pipeline = PretrainedPipeline("explain_document_dl", lang="en")

In [ ]:
import sparknlp

spark = sparknlp.start(spark32=True)
